## Building The Model
-----

### Setup:
---

In [27]:
# Import dependencies
from pathlib import Path
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
import pandas as pd
import tensorflow as tf

ModuleNotFoundError: No module named 'tensorflow'

In [28]:
# Import the Combined Data from /Wrangling_Data_Exports
wrangled_df = pd.read_csv("Wrangling_Data_Exports/Wrangled_Data.csv", index_col=0)

# Display Dataframe
wrangled_df.head(5)

,Year,Month,Day,Time (EST),Season Week,Venue,City,State,Weather Condition,Temperature (F),...,Home Team Pre-Game Season W/L Ratio,Home Team Pre-Game Season Avg Points For,Home Team Pre-Game Season Avg Points Against,Outcome,Away Team,Away Team Pre-Game Season W Streak,Away Team Pre-Game Season L Streak,Away Team Pre-Game Season W/L Ratio,Away Team Pre-Game Season Avg Points For,Away Team Pre-Game Season Avg Points Against
0,2022.0,September,25,14:25,3.0,State Farm Stadium,Glendale,Arizona,None (retractable roof closed),72.0,...,0.50,25.00,33.50,2.0,Los Angeles Rams,1.0,0.0,0.50,20.50,29.00
1,2022.0,November,6,16:05,9.0,State Farm Stadium,Glendale,Arizona,Sunny,76.0,...,0.38,22.75,26.25,2.0,Seattle Seahawks,3.0,0.0,0.62,26.25,24.88
2,2022.0,November,27,16:05,12.0,State Farm Stadium,Glendale,Arizona,None (retractable roof closed),72.0,...,0.36,21.82,26.91,2.0,Los Angeles Chargers,0.0,2.0,0.50,22.70,25.80
3,2022.0,December,12,20:15,14.0,State Farm Stadium,Glendale,Arizona,None (retractable roof closed),72.0,...,0.33,22.00,26.75,2.0,New England Patriots,0.0,2.0,0.50,20.75,18.83
4,2022.0,December,25,20:20,16.0,State Farm Stadium,Glendale,Arizona,Clear,69.0,...,0.29,20.86,26.57,2.0,Tampa Bay Buccaneers,0.0,2.0,0.43,17.64,20.57


### Prepare the Data:
---

In [29]:
# Create Target Array
targets= wrangled_df["Outcome"].astype(int).values

# Display Array
targets

array([2, 2, 2, 2, 2, 2, 1, 1, 2, 2, 1, 1, 2, 2, 1, 1, 1, 1, 1, 2, 1, 1,
       1, 2, 1, 1, 1, 2, 1, 2, 1, 1, 1, 2, 1, 1, 2, 1, 1, 1, 1, 1, 1, 2,
       1, 2, 2, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 1, 2, 1,
       2, 2, 1, 2, 1, 1, 1, 2, 2, 1, 3, 2, 2, 2, 2, 2, 2, 2, 1, 2, 1, 2,
       2, 2, 2, 2, 1, 2, 2, 2, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 2,
       1, 1, 2, 1, 1, 2, 2, 1, 1, 1, 2, 1, 2, 1, 2, 2, 1, 1, 1, 1, 2, 1,
       1, 1, 2, 1, 1, 1, 1, 2, 1, 1, 1, 1, 2, 1, 1, 1, 2, 2, 1, 2, 1, 2,
       1, 2, 1, 1, 2, 1, 2, 1, 1, 2, 3, 2, 1, 2, 2, 1, 2, 1, 2, 2, 1, 1,
       1, 1, 2, 1, 1, 2, 1, 2, 2, 1, 1, 2, 2, 1, 1, 1, 2, 1, 2, 2, 2, 2,
       1, 1, 1, 1, 2, 1, 2, 1, 1, 1, 2, 2, 2, 2, 1, 2, 2, 1, 2, 1, 2, 1])

In [30]:
# Create Features Dataframe
features_df = wrangled_df.drop(columns=["Outcome"])

# Display Dataframe
features_df.head(5)

,Year,Month,Day,Time (EST),Season Week,Venue,City,State,Weather Condition,Temperature (F),...,Home Team Pre-Game Season L Streak,Home Team Pre-Game Season W/L Ratio,Home Team Pre-Game Season Avg Points For,Home Team Pre-Game Season Avg Points Against,Away Team,Away Team Pre-Game Season W Streak,Away Team Pre-Game Season L Streak,Away Team Pre-Game Season W/L Ratio,Away Team Pre-Game Season Avg Points For,Away Team Pre-Game Season Avg Points Against
0,2022.0,September,25,14:25,3.0,State Farm Stadium,Glendale,Arizona,None (retractable roof closed),72.0,...,0.0,0.50,25.00,33.50,Los Angeles Rams,1.0,0.0,0.50,20.50,29.00
1,2022.0,November,6,16:05,9.0,State Farm Stadium,Glendale,Arizona,Sunny,76.0,...,1.0,0.38,22.75,26.25,Seattle Seahawks,3.0,0.0,0.62,26.25,24.88
2,2022.0,November,27,16:05,12.0,State Farm Stadium,Glendale,Arizona,None (retractable roof closed),72.0,...,1.0,0.36,21.82,26.91,Los Angeles Chargers,0.0,2.0,0.50,22.70,25.80
3,2022.0,December,12,20:15,14.0,State Farm Stadium,Glendale,Arizona,None (retractable roof closed),72.0,...,2.0,0.33,22.00,26.75,New England Patriots,0.0,2.0,0.50,20.75,18.83
4,2022.0,December,25,20:20,16.0,State Farm Stadium,Glendale,Arizona,Clear,69.0,...,4.0,0.29,20.86,26.57,Tampa Bay Buccaneers,0.0,2.0,0.43,17.64,20.57


In [31]:
# Git Dummies
dummies_features_df = pd.get_dummies(features_df)

# Display Dataframe
dummies_features_df.head(5)

,Year,Day,Season Week,Temperature (F),Home Team Pre-Game Season W Streak,Home Team Pre-Game Season L Streak,Home Team Pre-Game Season W/L Ratio,Home Team Pre-Game Season Avg Points For,Home Team Pre-Game Season Avg Points Against,Away Team Pre-Game Season W Streak,...,Away Team_New Orleans Saints,Away Team_New York Giants,Away Team_New York Jets,Away Team_Philadelphia Eagles,Away Team_Pittsburgh Steelers,Away Team_San Francisco 49ers,Away Team_Seattle Seahawks,Away Team_Tampa Bay Buccaneers,Away Team_Tennessee Titans,Away Team_Washington Commanders
0,2022.0,25,3.0,72.0,1.0,0.0,0.50,25.00,33.50,1.0,...,0,0,0,0,0,0,0,0,0,0
1,2022.0,6,9.0,76.0,0.0,1.0,0.38,22.75,26.25,3.0,...,0,0,0,0,0,0,1,0,0,0
2,2022.0,27,12.0,72.0,0.0,1.0,0.36,21.82,26.91,0.0,...,0,0,0,0,0,0,0,0,0,0
3,2022.0,12,14.0,72.0,0.0,2.0,0.33,22.00,26.75,0.0,...,0,0,0,0,0,0,0,0,0,0
4,2022.0,25,16.0,69.0,0.0,4.0,0.29,20.86,26.57,0.0,...,0,0,0,0,0,0,0,1,0,0


In [32]:
# Create an Array from the Features Dataframe with Dummies
features_array = dummies_features_df.values

features_array

array([[2.022e+03, 2.500e+01, 3.000e+00, ..., 0.000e+00, 0.000e+00,
        0.000e+00],
       [2.022e+03, 6.000e+00, 9.000e+00, ..., 0.000e+00, 0.000e+00,
        0.000e+00],
       [2.022e+03, 2.700e+01, 1.200e+01, ..., 0.000e+00, 0.000e+00,
        0.000e+00],
       ...,
       [2.022e+03, 2.700e+01, 1.200e+01, ..., 0.000e+00, 0.000e+00,
        0.000e+00],
       [2.022e+03, 1.000e+00, 1.700e+01, ..., 0.000e+00, 0.000e+00,
        0.000e+00],
       [2.022e+03, 8.000e+00, 1.800e+01, ..., 0.000e+00, 0.000e+00,
        0.000e+00]])

In [33]:
# Scale the Data

# Create a Scaler
scaler = StandardScaler()

# Fit the StandardScaler
features_scaler = scaler.fit(features_array)

# Scale the Features
scaled_features = features_scaler.transform(features_array)

scaled_features

array([[ 0.        ,  1.15283579, -1.19258603, ..., -0.18128389,
        -0.19425717, -0.16744367],
       [ 0.        , -1.03159467, -0.05019593, ..., -0.18128389,
        -0.19425717, -0.16744367],
       [ 0.        ,  1.38277583,  0.52099912, ..., -0.18128389,
        -0.19425717, -0.16744367],
       ...,
       [ 0.        ,  1.38277583,  0.52099912, ..., -0.18128389,
        -0.19425717, -0.16744367],
       [ 0.        , -1.60644479,  1.47299087, ..., -0.18128389,
        -0.19425717, -0.16744367],
       [ 0.        , -0.80165462,  1.66338922, ..., -0.18128389,
        -0.19425717, -0.16744367]])

In [ ]:
# Primary Component Analysis?

In [34]:
# Train/Test Split The Data
X_train, X_test, y_train, y_test = train_test_split(scaled_features ,targets,random_state=1)

### Prepare the Random Forest Model:
---

In [40]:
# Create The Model/Models
rf_model = RandomForestClassifier(random_state=1, n_estimators=1000).fit(X_train, y_train)

In [41]:
# Check Training and Testing Scores

print(f'Training Score: {rf_model.score(X_train, y_train)}')
print(f'Testing Score: {rf_model.score(X_test, y_test)}')

Training Score: 1.0
Testing Score: 0.5818181818181818


### Prepare the Nural Network Model:
---

In [ ]:
# Count features
features_count = len(X_train[0])
features_count

In [ ]:
# Define Nural Network Model

nn = tf.keras.models.Sequential()

# Input layer
nn.add(tf.keras.layers.Dense(units=4, activation="relu", input_dim=features_count))

# Second layer
nn.add(tf.keras.layers.Dense(units=2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

In [ ]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam",metrics=["accuracy"])

In [ ]:
# Train the model
fit_nn = nn.fit(X_train, y_train, epochs=100)

In [ ]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

### Train the Model:
---

In [ ]:
# Train the Model/Models

### Test the Model:
---

In [ ]:
# Test the Model/Models

### Analyze the Accuracy of the Model:
---

### Export the Trained Model:
---

In [ ]:
# Export the Trained Model to ../04-Creating_Applicaton/Building_Model_Exports
